In [3]:
from pymilvus import MilvusClient

#数据库名称
db_name = "cost_assistant_sh"
#集合名称  一个数据库内可能有多个不同名的集合 集合可以和数据库同名
cl_name = "cost_assistant_sh"


In [ ]:
# 连接到Milvus服务器，这里假设服务器本地运行，端口默认19530，若实际情况不同需修改 ，建议这里就定位到需要的数据库
client = MilvusClient(uri="http://10.193.41.29", port="19530",db_name=db_name) 
# 列出所有数据库，确认目标数据库是否存在
databases = client.list_databases() 
print(len(databases)) #数据库的数量

6


In [3]:

# 遍历每个collection名称并获取对应的collection对象
for database in databases:
    print(database ) 
    if(database==db_name):
        collections = client.list_collections()
        print(len(collections))
        break

print(len(collections))
# for collection in collections :
#     print(collection) 
collection =collections[0]
print(collection)

sepd_vector_store
cost_assistant_sh_BM25
cost_assistant_sh
1
1
cost_assistant_sh


In [5]:
# 查询集合中的数据
collection_name = cl_name # 替换为你的集合名称
filter_condition = 'name_property == "该设备为：500kV变压器，属于：一次设备，规格为：500kV,334MVA,单相,500/220/66,无励磁,三绕组自耦"'  # 替换为你的过滤条件
output_fields = ["name_property" ,"describ_dense"]  # 替换为你的字段名称

# 执行查询
res = client.query(
    collection_name=collection_name,
    filter=filter_condition,
    output_fields=output_fields,
    limit=3  # 限制返回的结果数量
)

# 打印查询结果
print(res)

data: ["{'describ_dense': [np.float32(-0.006548017), np.float32(0.023878329), np.float32(-0.042785533), np.float32(0.041701473), np.float32(0.021001427), np.float32(-0.014218672), np.float32(0.011657427), np.float32(0.08013745), np.float32(0.026170133), np.float32(0.02300958), np.float32(0.004172615), np.float32(-0.0071427696), np.float32(-0.0034473957), np.float32(-0.017091304), np.float32(-0.00828348), np.float32(-0.007815264), np.float32(0.008189925), np.float32(-0.030305753), np.float32(0.0296191), np.float32(-0.00038980838), np.float32(-0.030009044), np.float32(0.016146349), np.float32(0.008843948), np.float32(0.03218753), np.float32(-0.003568792), np.float32(0.0063475696), np.float32(-0.05444416), np.float32(-0.012470382), np.float32(0.03894844), np.float32(0.0075854505), np.float32(-0.026214778), np.float32(-0.029688984), np.float32(0.023864187), np.float32(0.011979978), np.float32(-0.010803895), np.float32(-0.022323264), np.float32(-0.04037944), np.float32(-0.0047815545), np.fl